In [3]:
from experiment import ResultsViewer
from IPython.display import Image
import torch
from scipy.io import savemat, loadmat
import os

In [4]:
res = ResultsViewer("experiments/experiment_1_server")

def lookup(results, *filters):
  r = []
  for res in results:
    skip = False
    for f in filters:
      if not f(res):
        skip = True
        break
    if not skip:
      r.append(res)
  return r

def save_to_matlab(res):
  os.makedirs("matlab_models", exist_ok=True)
  W = []
  V = []
  skip = []
  out2 = {}
  sd = torch.load(res['state_dict_path'])
  for i in range(res['model']['layers']):
    key = 'blocks.{}.{}.weight'
    w = sd[key.format(i, 'W')].numpy()
    v = sd[key.format(i, 'V')].numpy()  
    s = sd[key.format(i, 'skip_l')].numpy()  
    W.append(w)
    V.append(v)
    skip.append(s)
    out2[f'W_{i}'] = w
    out2[f'V_{i}'] = v
    out2[f'skip_{i}'] = s
  out = {"W": W, "V": V, "skip": skip}
  fname = f"D={res['data']['D']}_Term={res['model']['regularization_method']}_Lam={res['model']['regularization_lambda']}.mat"
  savemat("matlab_models/" + fname, out2)

In [6]:
r = lookup(
  res,
  lambda x: x['model']['regularization_method'] == 2,
  lambda x: x['model']['regularization_lambda'] == 0.001,
)
len(r)

1

In [10]:
os.path.dirname(r[0]['interpolation_sparsity_path'])

'experiments/experiment_1_server'